In [1]:
from sqlalchemy import create_engine, inspect
import pandas as pd
from urllib.parse import quote

#from sqlalchemy import  MetaData, Table, Column, Integer, String
from sqlalchemy_utils import database_exists, create_database
import psycopg2

In [2]:
import sys
import os

In [3]:
sys.path.append(os.path.join(os.path.dirname('__file__'), 'models'))
sys.path.append(os.path.join(os.path.dirname('__file__'), 'scripts'))
sys.path.append(os.path.join(os.path.dirname('__file__'), 'config'))

In [4]:
from config.params import DEBUG

In [5]:
from config.dbsecret import *

In [7]:
from sqlalchemy import create_engine
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

db_name = DB_NAME
user = DB_USER
password = DB_PASSWORD
host = DB_HOST
port = DB_PORT

# Step 1: Connect to PostgreSQL default database
try:
    # Connect to the default 'postgres' database
    conn = psycopg2.connect(dbname='postgres', user=user, password=password, host=host, port=port)
    conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)  # Allows database creation
    cursor = conn.cursor()
    
    # Check if the database already exists
    cursor.execute(f"SELECT 1 FROM pg_database WHERE datname = '{db_name}';")
    exists = cursor.fetchone()
    
    # Create the database if it does not exist
    if not exists:
        cursor.execute(f"CREATE DATABASE {db_name};")
        print(f"Database '{db_name}' created successfully.")
    else:
        print(f"Database '{db_name}' already exists.")
    
    cursor.close()
    conn.close()
except Exception as e:
    print(f"Error creating database: {e}")


Database 'pare' created successfully.


In [9]:
# SQLite connection
sqlite_engine = create_engine('sqlite:///pare.db')

# PostgreSQL connection to the new database
postgres_engine = create_engine(f'postgresql://{user}:{quote(password)}@{host}:{port}/{db_name}')

# Use SQLAlchemy's inspector to get the list of table names
inspector = inspect(sqlite_engine)
table_names = inspector.get_table_names()

# Load data from SQLite, then write to PostgreSQL
for table_name in table_names:
    df = pd.read_sql_table(table_name, sqlite_engine)
    df.to_sql(table_name, postgres_engine, if_exists='replace', index=False)
    print(f"Table '{table_name}' migrated successfully.")

Table 'activities' migrated successfully.
Table 'communes' migrated successfully.
Table 'communities' migrated successfully.
Table 'departments' migrated successfully.
Table 'participants' migrated successfully.
Table 'partners' migrated successfully.
Table 'sales' migrated successfully.
Table 'sections' migrated successfully.
Table 'temp_communities' migrated successfully.
